# Introduction

In this tutorial, we discuss the overall flow of TDAstro and how to use it to run simulations. The goal is to get a new user started and allow them to explore the package.

Later tutorials cover topics in more depth, including:
  - Sampling Parameters (sampling.ipynb)
  - Adding new source types (adding_sources.ipynb)
  - Add new effect types (addings_effects.ipynb)
  - Working directly with passbands (passband-demo.ipynb)
  - Working directly with OpSims (opsim_notebook.ipynb)

## Program Flow

TDAstro generates synthetic light curves using the flow shown in the illustration below. A `PhysicalModel` (the model of a source) and information about the parameter distributions is used to sample fake sources. These are combined with information from an `OpSim` (or other survey information) to generate sample flux densities at a given set of times and wavelengths (or passbands), accounting for effects such as redshift. The simulator also applies other relevant effects to the rest frame flux densities (e.g. dust extinction) and the observer frame flux densities (detector noise). At the end the code outputs a series of samples.

In [ ]:
from IPython.display import Image

Image("../_static/tdastro-intro.png")

## Sources

All light curves are generated from source objects that are a subclass of the `PhysicalObject` class. These source objects provide mechanisms for:
  - Sampling their parameters from given distributions,
  - Generating flux densities at given times and wavelengths (or passbands), and
  - Applying noise and other effects to the observations.

Each "sample" of the data consists of a new sampling of the source's parameters and a generation of flux densities from those parameters. Thus, when a user generates a hundred samples, they are generating 100 light curves from 100 sample sources.

We can demonstrate this using a toy model that generates fluxes using a sin wave. The model's three parameters (`brightness`, `frequency`, and `t0`) are chosen for each source from uniform distributions. For more information on how to how to define the parameter setting, see the `sampling.ipynb` notebook.

In [ ]:
from tdastro.math_nodes.np_random import NumpyRandomFunc
from tdastro.sources.basic_sources import SinWaveSource

source_model = SinWaveSource(
    brightness=NumpyRandomFunc("uniform", low=100.0, high=200.0),
    frequency=NumpyRandomFunc("uniform", low=1.0, high=5.0),
    t0=NumpyRandomFunc("uniform", low=0.0, high=10.0),
)

We can manually evalute a source model using the `evaluate()` function where we provide the wavelengths and times to sample:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

times = np.arange(100.0)
wavelengths = np.array([7000.0])
fluxes = source_model.evaluate(times, wavelengths)

plt.plot(times, fluxes)
plt.xlabel("Time")
plt.ylabel("Flux")
plt.show()

The power of the simulation software is that we can generate a large number of light curves from a distribution of sources. We start by using a `PhysicalModel` object's `sample_parameters` function to sample the parameters can create this distribution of source object. 

Let's start with generating 5 source objects. We save the samples in a `GraphState` object. Users will not need to deal with this object directly, but it can be used to peak at the underlying parameters.

In [ ]:
state = source_model.sample_parameters(num_samples=3)
print(state)

Most users will not need to interact directly with the `GraphState` object, but at a very high level it can be viewed as a nested dictionary where parameters are indexed by two levels. First, the node name or identifier tells the code which part of the simulation the parameter belongs to. This level of identification is necessary to allow different stages to use parameters with the same name. Second, the parameter name defines the actual parameter name.

Each parameter name corresponds to a list of samples. These sampled together so that the i-th entires of each parameter represent a single, mutually consistent sampling of parameter space. For a lot more detail see the `GraphState` section in the `sampling.ipynb` notebook. For now it is sufficient to know that `state` is tracking the sampled parameters.

By passing the sampled state into `evaluate()` we can generate multiple light curves (one for each source) at once:

In [ ]:
fluxes = source_model.evaluate(times, wavelengths, state)

plt.plot(times, fluxes[0, :], color="blue")
plt.plot(times, fluxes[1, :], color="green")
plt.plot(times, fluxes[2, :], color="red")
plt.xlabel("Time")
plt.ylabel("Flux")
plt.show()